# A notebook to perform a genetic algorithm for conformer analysis. This is currently being implemented on just molecules.

In [1]:
import os
import sys
import logging
FORMAT = "%(filename)s:%(lineno)d %(funcName)s %(levelname)s %(message)s"
logging.basicConfig(format=FORMAT, level=logging.INFO)

import re
import imp
import itertools
import random
import numpy as np
from numpy import array
import pandas as pd
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns


# do this before we have a chance to import openbabel!
import rdkit, rdkit.Chem.rdDistGeom, rdkit.DistanceGeometry

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import rdBase

import py3Dmol

from rmgpy.molecule import Molecule
from rmgpy.species import Species
from rmgpy.reaction import Reaction


from multi_molecule import *
from multi_reaction import *

from ase.calculators.morse import * #chosing this calculator for now because it's fast
from ase.calculators.dftb import *
from ase.calculators.lj import *
from ase.calculators.emt import *

from copy import deepcopy

/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
thermo.py:760 loadLibraries INFO Loading thermodynamics library from CBS_QB3_1dHR.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:760 loadLibraries INFO Loading thermodynamics library from KlippensteinH2O2.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:760 loadLibraries INFO Loading thermodynamics library from primaryThermoLibrary.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:760 loadLibraries INFO Loading thermodynamics library from thermo_DFT_CCSDTF12_BAC.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:774 loadGroups INFO Loading thermodynamics group database from /Users/nathan/Code/RMG-database/input/

In [2]:
mol= Multi_Molecule("COCCNC")
mol.view_mol()

In [3]:
possible_dihedrals = np.arange(0, 360+30, 30)


calc = EMT()
mol.ase_molecule.set_calculator(calc)

population_size = 20

population = []

for i in range(population_size):
    mol_copy = deepcopy(mol)
    dihedrals = []
    
    for torsion in mol_copy.torsions:
        dihedral = np.random.choice(possible_dihedrals)
        dihedrals.append(dihedral)
        i,j,k,l = torsion.indices
        RHS = torsion.RHS
        
        mol_copy.ase_molecule.set_dihedral(i,j,k,l, dihedral)
    
    mol_copy.update_geometry_from_ase_mol()
    
    
    e = mol_copy.ase_molecule.get_potential_energy()
    
    population.append( [e] + dihedrals )
    
df = pd.DataFrame(population)
columns = ["Energy"]
for i in range(len(mol.torsion_list)):
    columns = columns +["Torsion " + str(i)] 
df.columns = columns
df

Energy  Torsion 0  Torsion 1  Torsion 2
0   98.092498        210        180          0
1   50.577988        180        330        180
2   65.907376        330        180        210
3   64.319452        270        360        330
4   87.781021        330        240          0
5   59.546410          0        300         30
6   37.297176        150        330         60
7   54.942376        180        150        210
8   11.606672        150        120         60
9   17.682171        150        120        360
10  24.729793        150        270        210
11  50.490059        180        360         90
12  84.910956        210        240        150
13  58.467763        300        330         90
14  12.553726        120        270        270
15  25.537126        360        120        360
16  54.614284        300        360        270
17  97.319727        210          0        240
18   7.978008        120        120        300
19  16.380958        360         90        120

In [4]:
df = df.sort("Energy")
df

/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


Energy  Torsion 0  Torsion 1  Torsion 2
18   7.978008        120        120        300
8   11.606672        150        120         60
14  12.553726        120        270        270
19  16.380958        360         90        120
9   17.682171        150        120        360
10  24.729793        150        270        210
15  25.537126        360        120        360
6   37.297176        150        330         60
11  50.490059        180        360         90
1   50.577988        180        330        180
16  54.614284        300        360        270
7   54.942376        180        150        210
13  58.467763        300        330         90
5   59.546410          0        300         30
3   64.319452        270        360        330
2   65.907376        330        180        210
12  84.910956        210        240        150
4   87.781021        330        240          0
17  97.319727        210          0        240
0   98.092498        210        180          0

## We now have a population of 20 molecules that have been sorted based on their energies.

### Now we need to set the following:

* Chance of cross-over
* Chance of mutation
* Percentage of the population that survives

`I honestly don't know which numbers to select, but here are random guesses`

In [5]:
crossover_probability = 0.5
mutation_probability = 0.1
survival_percent = 0.5

In [6]:
top_population_size = int(survival_percent * float(population_size))
bottom_population_size = int((1-survival_percent) * float(population_size))

In [7]:
dff = df.iloc[:top_population_size,:]


In [8]:
population = []
for i in range(bottom_population_size):
    m,f = random.sample(np.arange(top_population_size), 2)
    
    print "The `male` is {0} and the `female` is {1}".format(m,f)
    
    
    
    mol_copy = deepcopy(mol)
    dihedrals = []
    
    for i, torsion in enumerate(mol_copy.torsions):
        
        print i
        
        mutation = random.random()
        
        crossover = random.random()
        
        if mutation > mutation_probability:
            if crossover > crossover_probability:
                dihedral = dff.iloc[m, i+1]
                print "The male torsion was chosen. The dihedral is {}".format(dihedral)
            else:
                dihedral = dff.iloc[f, i+1]
                print "The male torsion was chosen. The dihedral is {}".format(dihedral)
        else:
            dihedral = np.random.choice(possible_dihedrals)
            print "MUTATION. The dihedral is {}".format(dihedral)
        
        print
        dihedrals.append(dihedral)
        i,j,k,l = torsion.indices
        RHS = torsion.RHS
        
        mol_copy.ase_molecule.set_dihedral(i,j,k,l, dihedral)
    
    mol_copy.update_geometry_from_ase_mol()
    
    
    e = mol_copy.ase_molecule.get_potential_energy()
    
    population.append( [e] + dihedrals )
    
dfff = pd.DataFrame(population)
columns = ["Energy"]
for i in range(len(mol.torsion_list)):
    columns = columns +["Torsion " + str(i)] 
dfff.columns = columns
dfff

The `male` is 6 and the `female` is 5
0
The male torsion was chosen. The dihedral is 150.0

1
The male torsion was chosen. The dihedral is 270.0

2
The male torsion was chosen. The dihedral is 210.0

The `male` is 4 and the `female` is 8
0
The male torsion was chosen. The dihedral is 150.0

1
The male torsion was chosen. The dihedral is 360.0

2
The male torsion was chosen. The dihedral is 360.0

The `male` is 9 and the `female` is 0
0
The male torsion was chosen. The dihedral is 120.0

1
The male torsion was chosen. The dihedral is 330.0

2
The male torsion was chosen. The dihedral is 300.0

The `male` is 3 and the `female` is 5
0
The male torsion was chosen. The dihedral is 360.0

1
The male torsion was chosen. The dihedral is 270.0

2
MUTATION. The dihedral is 120

The `male` is 4 and the `female` is 2
0
The male torsion was chosen. The dihedral is 120.0

1
The male torsion was chosen. The dihedral is 270.0

2
The male torsion was chosen. The dihedral is 270.0

The `male` is 2 and t

Energy  Torsion 0  Torsion 1  Torsion 2
0  24.729793      150.0      270.0      210.0
1  41.488144      150.0      360.0      360.0
2  33.671249      120.0      330.0      300.0
3  19.397498      360.0      270.0      120.0
4  12.553726      120.0      270.0      270.0
5   8.553941      120.0      120.0      330.0
6  19.397498      360.0      270.0      120.0
7  10.461134      120.0      120.0      360.0
8  37.297176      150.0      330.0       60.0
9  13.964704      120.0       90.0      120.0

In [9]:
new_population = dff.append(dfff)
new_population.sort("Energy")

/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


Energy  Torsion 0  Torsion 1  Torsion 2
18   7.978008      120.0      120.0      300.0
5    8.553941      120.0      120.0      330.0
7   10.461134      120.0      120.0      360.0
8   11.606672      150.0      120.0       60.0
14  12.553726      120.0      270.0      270.0
4   12.553726      120.0      270.0      270.0
9   13.964704      120.0       90.0      120.0
19  16.380958      360.0       90.0      120.0
9   17.682171      150.0      120.0      360.0
3   19.397498      360.0      270.0      120.0
6   19.397498      360.0      270.0      120.0
0   24.729793      150.0      270.0      210.0
10  24.729793      150.0      270.0      210.0
15  25.537126      360.0      120.0      360.0
2   33.671249      120.0      330.0      300.0
6   37.297176      150.0      330.0       60.0
8   37.297176      150.0      330.0       60.0
1   41.488144      150.0      360.0      360.0
11  50.490059      180.0      360.0       90.0
1   50.577988      180.0      330.0      180.0

## Sweet, we now have set up a basic genetic algorithm for a molecule 

### Now to try this for multiple generations automatically

In [28]:
# First, making the first generation

possible_dihedrals = np.arange(0, 360, 30)


calc = EMT()
mol.ase_molecule.set_calculator(calc)

population_size = 50

population = []

for i in range(population_size):
    mol_copy = deepcopy(mol)
    dihedrals = []
    
    for torsion in mol_copy.torsions:
        dihedral = np.random.choice(possible_dihedrals)
        dihedrals.append(dihedral)
        i,j,k,l = torsion.indices
        RHS = torsion.RHS
        
        mol_copy.ase_molecule.set_dihedral(i,j,k,l, dihedral)
    
    mol_copy.update_geometry_from_ase_mol()
    
    
    e = mol_copy.ase_molecule.get_potential_energy()
    
    population.append( [e] + dihedrals )
    
df = pd.DataFrame(population)
columns = ["Energy"]
for i in range(len(mol.torsion_list)):
    columns = columns +["Torsion " + str(i)] 
df.columns = columns
df = df.sort("Energy")
df

/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/ipykernel/__main__.py:37: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


Energy  Torsion 0  Torsion 1  Torsion 2
14    8.845900        240        210        270
46    9.220911        180        240        210
25   14.635920        240        210         90
34   15.437375        270        270        180
11   16.147629        270        210        120
31   16.948139        150        300        210
27   16.950132        240        270         30
1    17.237522        150        180        150
12   17.643663        150        180         60
42   18.076980        210         30         60
8    18.076980        210         30         60
39   18.802944         30        240        300
16   19.306463        240        180         60
35   19.907088         30        330         90
29   20.111000        270        300         90
0    20.557689        270          0        300
24   20.734035        120        300        120
49   22.649607        180         30        300
28   23.943492        300        210        120
19   26.206762        150         30        180
36   26.643414        300        330        270
7    29.690625         60        300        180
30   30.363457          0        150        240
3    30.733579        240        150          0
48   30.904001        300         30        120
26   32.647058         60        180         90
40   32.940454         60          0        240
4    33.287905        270         30        330
45   34.274275        300        120        300
43   35.200803        300        300        330
47   36.016321          0        330          0
2    36.967875        300         60         30
17   38.873958        300         60        300
44   44.419231        210         90        270
15   45.169185        150         90        120
10   47.216585          0         30        120
23   52.815360         30         90        240
37   58.537930         60         60        330
6    60.128400          0        270         30
41   63.214813          0         90        270
20   67.151841          0        180        330
33   68.687976         90        300        120
38   70.803641         90         30        120
9    81.330272        120        300        150
32   84.005814        330        150         90
21   85.366345         90         30        150
13   87.195980        330        120        180
22   95.749826        330         60        270
18  106.715855         90         90        210
5   112.111100        330         90        240

Setting the percentages and probabilities and selecting the top population

In [29]:
crossover_probability = 0.5
mutation_probability = 0.5
survival_percent = 0.5

top_population_size = int(survival_percent * float(population_size))
bottom_population_size = int((1-survival_percent) * float(population_size))


In [30]:
generations = 100
mol_copy = deepcopy(mol)

for generation in range(generations):
    print "This is the {}th generation".format(generation)
    top_population = df.iloc[:top_population_size,:]
    population = []
    for i in range(population_size):
        mf = random.sample(np.arange(top_population_size), 2)
        m = max(mf)
        f = min(mf)

        print "The `male` is {0} and the `female` is {1}".format(m,f)



        #mol_copy = deepcopy(mol)
        dihedrals = []

        for i, torsion in enumerate(mol_copy.torsions):

            print i

            mutation = random.random()

            crossover = random.random()

            if mutation > mutation_probability:
                if crossover > crossover_probability:
                    dihedral = top_population.iloc[m, i+1]
                    print "The male torsion was chosen. The dihedral is {}".format(dihedral)
                else:
                    dihedral = top_population.iloc[f, i+1]
                    print "The male torsion was chosen. The dihedral is {}".format(dihedral)
            else:
                dihedral = np.random.choice(possible_dihedrals)
                print "MUTATION. The dihedral is {}".format(dihedral)

            print
            dihedrals.append(dihedral)
            i,j,k,l = torsion.indices
            RHS = torsion.RHS

            mol_copy.ase_molecule.set_dihedral(i,j,k,l, dihedral)

        mol_copy.update_geometry_from_ase_mol()


        e = mol_copy.ase_molecule.get_potential_energy()

        population.append( [e] + dihedrals )

    df = pd.DataFrame(population)
    columns = ["Energy"]
    for i in range(len(mol.torsion_list)):
        columns = columns +["Torsion " + str(i)] 
    df.columns = columns
    """
    df = top_population.append(new_population)"""
    df = df.sort("Energy")
    print df.iloc[0:5,:]
    print "~~~~~~~~~~~~~~~~~~~~~~~~~~~~"

This is the 0th generation
The `male` is 12 and the `female` is 5
0
MUTATION. The dihedral is 90

1
MUTATION. The dihedral is 300

2
MUTATION. The dihedral is 120

The `male` is 14 and the `female` is 7
0
MUTATION. The dihedral is 300

1
The male torsion was chosen. The dihedral is 180.0

2
The male torsion was chosen. The dihedral is 90.0

The `male` is 14 and the `female` is 4
0
MUTATION. The dihedral is 0

1
The male torsion was chosen. The dihedral is 300.0

2
The male torsion was chosen. The dihedral is 120.0

The `male` is 21 and the `female` is 1
0
The male torsion was chosen. The dihedral is 60.0

1
MUTATION. The dihedral is 300

2
MUTATION. The dihedral is 0

The `male` is 22 and the `female` is 17
0
The male torsion was chosen. The dihedral is 0.0

1
The male torsion was chosen. The dihedral is 30.0

2
MUTATION. The dihedral is 120

The `male` is 23 and the `female` is 16
0
The male torsion was chosen. The dihedral is 240.0

1
The male torsion was chosen. The dihedral is 300.

/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/ipykernel/__main__.py:60: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)



The `male` is 22 and the `female` is 2
0
The male torsion was chosen. The dihedral is 210.0

1
The male torsion was chosen. The dihedral is 30.0

2
The male torsion was chosen. The dihedral is 240.0

The `male` is 16 and the `female` is 3
0
MUTATION. The dihedral is 330

1
MUTATION. The dihedral is 300

2
MUTATION. The dihedral is 240

The `male` is 23 and the `female` is 2
0
MUTATION. The dihedral is 180

1
The male torsion was chosen. The dihedral is 210.0

2
MUTATION. The dihedral is 150

The `male` is 15 and the `female` is 11
0
The male torsion was chosen. The dihedral is 240.0

1
MUTATION. The dihedral is 150

2
The male torsion was chosen. The dihedral is 330.0

The `male` is 17 and the `female` is 12
0
The male torsion was chosen. The dihedral is 240.0

1
The male torsion was chosen. The dihedral is 180.0

2
The male torsion was chosen. The dihedral is 90.0

The `male` is 9 and the `female` is 4
0
MUTATION. The dihedral is 300

1
The male torsion was chosen. The dihedral is 27

In [32]:
df

Energy  Torsion 0  Torsion 1  Torsion 2
24  12.356083      180.0      150.0      300.0
46  12.889919      180.0      150.0      210.0
11  13.919338      180.0      180.0      120.0
17  14.637376      150.0      300.0      300.0
41  14.736952      150.0      270.0      120.0
28  14.740586      150.0      180.0      300.0
3   14.763134      240.0      270.0       60.0
19  14.854085      240.0      210.0      150.0
18  15.153660      150.0      180.0      300.0
36  15.380049      180.0      180.0      120.0
0   15.416906      180.0      300.0      210.0
23  15.644316      150.0        0.0      300.0
2   15.755522      180.0      180.0      330.0
29  16.071411      210.0      330.0      240.0
15  16.093841      180.0      180.0      120.0
39  16.212367      180.0      180.0      120.0
20  16.326278      270.0      150.0      270.0
16  16.449411      180.0        0.0      180.0
35  16.608916      270.0      240.0       60.0
26  17.175112      150.0      330.0        0.0
12  17.194652      270.0      270.0       60.0
13  17.313433      150.0      300.0      180.0
5   19.450355      240.0      300.0      210.0
47  19.535932      270.0      150.0      120.0
37  19.837806      240.0      330.0      120.0
22  19.946081      240.0      180.0      150.0
31  20.110217       30.0      150.0      120.0
49  20.156886      210.0      300.0       30.0
33  20.611557      270.0      180.0      150.0
40  20.920815      270.0        0.0      150.0
44  20.946059       30.0       30.0       90.0
45  24.221527      210.0        0.0      330.0
25  26.365909      180.0      120.0        0.0
38  26.552969      180.0       60.0      210.0
10  26.748462      270.0      150.0      210.0
48  27.922902       60.0      270.0        0.0
7   28.751899      300.0        0.0      300.0
30  30.657935        0.0      180.0       30.0
8   30.691403      300.0      300.0      330.0
14  31.508525        0.0      150.0      210.0
1   31.542050        0.0      270.0      300.0
6   31.758966      270.0       60.0      150.0
4   43.081321        0.0       60.0      120.0
32  46.266784      150.0       90.0      180.0
21  47.433728      150.0       90.0        0.0
9   51.703032      270.0       90.0      120.0
42  65.109507       30.0        0.0      120.0
34  73.122978       90.0      240.0      270.0
43  73.980039      210.0      300.0      210.0
27  82.862218      330.0      270.0       30.0

## Okay, we have something. So now let's compare this to the brute force method

In [14]:
mol

<multi_molecule.Multi_Molecule instance at 0x10445f998>

In [15]:

mol.ase_molecule.set_calculator(calc)

# Getting the torsion combos
torsion_list = mol.torsions
torsion_angles = np.arange(0, 360,30) ### You can change the degree step size
torsion_combos = list( itertools.combinations_with_replacement( torsion_angles, len(torsion_list)) )
if len(torsion_list) != 1:
    torsion_combos = list(
        set(
            torsion_combos + 
            list(itertools.combinations_with_replacement( 
                torsion_angles[::-1], len(torsion_list)
            ))))

    results = []
    
# Calculating the potential energy for each conformation
for combo in torsion_combos:
    geo = zip(torsion_list, combo)
    #print geo
    for torsion in geo:
        tor = torsion[0]
        #print tor
        angle = torsion[1]
        
        i,j,k,l = tor.indices
        RHS =  tor.RHS
        mol.ase_molecule.set_dihedral(a1 = i,
                                a2 = j, 
                                a3 = k, 
                                a4 = l, 
                                angle= float(angle), 
                                indices=RHS)
    mol.update_geometry_from_ase_mol()
    results.append([mol.ase_molecule.get_potential_energy()] + list(combo) )

# Creating a dataframe of the results
brute_force = pd.DataFrame(results)
columns = ["Energy"]
for i in range(len(torsion_list)):
    columns = columns + ["Torsion " + str(i)] 

brute_force.columns = columns

brute_force.sort("Energy")

/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/ipykernel/__main__.py:46: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


Energy  Torsion 0  Torsion 1  Torsion 2
371    5.978668          0        270        270
3      5.981034        330        270        120
595    5.985056        330        270        270
18     5.985639          0        240        270
75     5.987366        330        300        270
99     5.987848        330        300        150
239    5.987863        330        270        150
481    5.988702        330        270        180
517    5.989538          0        240        240
294    5.990343        330        300        180
35     5.992723        330        270        210
310    5.993340        330        270        240
17     5.995013        330        300        210
465    5.995664        330        300        240
65     5.997715          0        210        210
579    5.997789        330        240        120
511    5.998155          0        210        240
362    5.998460        330        240        180
618    5.998761        330        240        150
682    5.998988         30        240        240
384    5.999326          0        210        270
425    6.002378        330        240        210
515    6.002852         30        210        240
176    6.003003        330        240        240
117    6.003101         30        210        210
462    6.003912        300        270        270
641    6.008995        300        270        180
195    6.009382        300        270        150
49     6.009628        300        270        120
581    6.009775        330        210        180
..          ...        ...        ...        ...
522   51.453945        330         30          0
194   51.689181          0          0        300
6     56.297004         60        300        330
672   58.557140        330         60          0
551   58.934844          0         60        330
488   59.657484        330        330         90
305   60.033710          0          0        330
556   60.569927          0        150        330
325   64.636928          0          0         60
216   64.857176          0          0         30
699   65.864528          0          0          0
559   67.192430        330        330         60
233   68.104756        330        150          0
31    68.225872         30         60        270
487   70.636030         30        150        300
182   74.934569         60        330        330
38    76.620082        330        120          0
100   81.747833         30         30        330
252   82.232487         30         60        330
316   86.501433        330        330         30
61    88.115535         60        300        300
674   89.770197        330        180         30
179  102.894957          0         60        300
190  108.437893         30        120        300
82   120.417801        300         30         30
20   121.836864        270         30         30
243  128.352137        300         60         60
261  144.069203         30         30        300
531  182.904353          0         30        330
187  202.319786        330        120         60

[716 rows x 4 columns]

Hmmm... we keep running into a local minimum... I need help with this